# Module 3: Statistics

**Python for Business | FinHub**

---

## What You'll Build

![Module 3 Preview](images/module3_preview.png)

## By the End of This Module

You'll be able to:

| Skill | Example |
|-------|--------|
| Calculate correlation | "How do AAPL and MSFT move together?" → 0.63 |
| Run a regression | Predict NVDA returns from market returns |
| Interpret results | "NVDA has beta = 2.0 (moves 2x the market)" |
| Evaluate model fit | R² = 0.58 means market explains 58% of variance |

**The tool:** scikit-learn — Python's machine learning library.

**The punchline:** Predicting stock returns is *hard*. We'll build a model, see it fail, and learn why that's actually an important result in finance.

### Download the Data

Before running the code below, you need to download the course data files.

1. Go to the course data repository (provided by your instructor)
2. Download `stock_prices.csv` and `market_data.csv`
3. Place them in the `data/` folder in this project

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression       # OLS regression
from sklearn.metrics import r2_score, mean_squared_error  # Model evaluation

# Load and prepare data
df = pd.read_csv("data/stock_prices.csv")            # Load stock data
df["date"] = pd.to_datetime(df["date"])                 # Convert to datetime
df = df.sort_values(["ticker", "date"]).reset_index(drop=True)  # Sort by stock, then date
df["return"] = df.groupby("ticker")["adj_close"].pct_change()  # Daily returns per stock

print(f"Loaded {len(df):,} rows")
print(f"Stocks: {df['ticker'].unique().tolist()}")

---

## 1. Correlation: Do Stocks Move Together?

Correlation measures how two variables move in relation:
- **+1**: Perfect positive (both go up together)
- **0**: No relationship
- **-1**: Perfect negative (one up, other down)

In [ ]:
# Pivot: each column becomes one stock's returns
returns = df.pivot(index="date", columns="ticker", values="return")  # Reshape: dates as rows, tickers as columns
returns.head()

In [ ]:
# Correlation matrix
corr = returns.corr().round(2)                          # Pairwise correlations, rounded

# Visualize as heatmap
fig, ax = plt.subplots(figsize=(8, 6))                  # Create figure
im = ax.imshow(corr, cmap="RdYlGn", vmin=-1, vmax=1)    # Color grid: red=neg, green=pos

ax.set_xticks(range(len(corr.columns)))                 # X-axis tick positions
ax.set_yticks(range(len(corr.columns)))                 # Y-axis tick positions
ax.set_xticklabels(corr.columns)                        # X-axis labels (tickers)
ax.set_yticklabels(corr.columns)                        # Y-axis labels (tickers)

for i in range(len(corr)):                              # Loop through rows
    for j in range(len(corr)):                          # Loop through columns
        ax.text(j, i, f"{corr.iloc[i, j]:.2f}",         # Add correlation value as text
                ha="center", va="center", fontsize=8)

plt.colorbar(im, label="Correlation")                   # Add color scale
plt.title("Stock Return Correlations")
plt.tight_layout()
plt.show()

print("Tech stocks (AAPL, MSFT, GOOGL, NVDA, META) are highly correlated.")
print("JNJ and XOM are less correlated with tech — diversification benefit.")

### ✏️ Checkpoint: Find the Least Correlated Pair

Which two stocks have the lowest correlation? (Hint: look at the heatmap or the correlation matrix)

In [ ]:
# Your code here


---

## 2. Regression: The Market Model

Correlation tells us *if* variables move together. **Regression** tells us *by how much*.

The **market model** is fundamental in finance:

$$R_{stock} = \alpha + \beta \cdot R_{market} + \epsilon$$

- **Alpha (α)**: Stock's return when market is flat
- **Beta (β)**: How much the stock moves when the market moves 1%
  - β = 1: Moves with market
  - β > 1: More volatile than market
  - β < 1: Less volatile than market

In [ ]:
# Load real market data (S&P 500 via SPY ETF)
market_df = pd.read_csv("data/market_data.csv")      # Load market data
market_df["date"] = pd.to_datetime(market_df["date"])   # Convert to datetime

# Get SPY returns
spy = market_df[market_df["ticker"] == "SPY"].set_index("date")["close_px"]  # SPY prices
spy_return = spy.pct_change()                           # SPY daily returns
spy_return.name = "market"                              # Rename for clarity

# Add market returns to our returns dataframe
returns = returns.join(spy_return)                      # Join on date index

# Prepare NVDA vs market data
data = returns[["NVDA", "market"]].dropna()             # Keep only NVDA and market, drop NaN
X = data["market"].values.reshape(-1, 1)                # X must be 2D for sklearn
y = data["NVDA"].values                                 # y is 1D array

print(f"Using SPY as market proxy")
print(f"Training on {len(X)} observations")

In [ ]:
# Fit the regression
model = LinearRegression()                              # Create empty model
model.fit(X, y)                                         # Fit: find best alpha and beta

# Results are stored in the model object
print("=" * 50)
print("MODEL RESULTS: NVDA vs Market")
print("=" * 50)
print(f"Alpha (intercept):  {model.intercept_:.6f}")    # model.intercept_ = alpha
print(f"Beta (slope):       {model.coef_[0]:.4f}")      # model.coef_ = [beta]
print(f"")
print(f"Interpretation:")
print(f"  • When market is flat, NVDA averages {model.intercept_:.4%} daily")
print(f"  • When market moves 1%, NVDA moves {model.coef_[0]:.2f}%")
print(f"  • NVDA is {'more' if model.coef_[0] > 1 else 'less'} volatile than the market")

In [ ]:
# Evaluate: How good is the fit?
y_pred = model.predict(X)                               # Predicted NVDA returns
r2 = r2_score(y, y_pred)                                # R² = variance explained
rmse = np.sqrt(mean_squared_error(y, y_pred))           # RMSE = avg prediction error

print(f"R-squared: {r2:.4f}")
print(f"  → Market explains {r2:.1%} of NVDA's daily return variance")
print(f"  → The remaining {1-r2:.1%} is stock-specific")
print(f"")
print(f"RMSE: {rmse:.4f} ({rmse:.2%} average prediction error)")

In [ ]:
# Visualize the regression
fig, ax = plt.subplots(figsize=(8, 6))                  # Create figure

ax.scatter(X, y, alpha=0.3, s=10, label="Actual")       # Scatter: each dot = one day
ax.plot(X, y_pred, color="red", linewidth=2,            # Red line = regression fit
        label=f"Fit: y = {model.intercept_:.4f} + {model.coef_[0]:.2f}x")

ax.axhline(0, color="gray", linestyle="--", alpha=0.5)  # Horizontal line at 0
ax.axvline(0, color="gray", linestyle="--", alpha=0.5)  # Vertical line at 0
ax.set_xlabel("Market Return")
ax.set_ylabel("NVDA Return")
ax.set_title(f"NVDA vs Market (R² = {r2:.3f})")
ax.legend()
plt.tight_layout()
plt.show()

### Calculate Beta for All Stocks

In [ ]:
# Calculate beta for every stock
results = []                                            # Store results here

for ticker in df["ticker"].unique():                    # Loop through each stock
    stock_ret = returns[ticker].dropna()                # This stock's returns
    mkt_ret = returns["market"].loc[stock_ret.index]    # Matching market returns
    
    X = mkt_ret.values.reshape(-1, 1)                   # Reshape for sklearn
    y = stock_ret.values                                # Stock returns
    
    model = LinearRegression().fit(X, y)                # Fit regression
    y_pred = model.predict(X)                           # Get predictions
    
    results.append({                                    # Save results
        "ticker": ticker,
        "alpha": model.intercept_,
        "beta": model.coef_[0],
        "r_squared": r2_score(y, y_pred)
    })

beta_df = pd.DataFrame(results).sort_values("beta", ascending=False)  # Sort by beta
beta_df

In [ ]:
# Visualize betas
colors = ["green" if b > 1 else "blue" for b in beta_df["beta"]]  # Green if beta > 1

plt.figure(figsize=(10, 5))
plt.barh(beta_df["ticker"], beta_df["beta"], color=colors)  # Horizontal bar chart
plt.axvline(1, color="black", linestyle="--", label="Market (β=1)")  # Reference line
plt.xlabel("Beta")
plt.title("Stock Betas (Market Sensitivity)")
plt.legend()
plt.tight_layout()
plt.show()

print("Green = more volatile than market, Blue = less volatile")

### ✏️ Checkpoint: Defensive vs Aggressive

Which stock is the most "defensive" (lowest beta)? Which is most "aggressive" (highest beta)?

In [ ]:
# Your code here


---

## 3. Multiple Regression: Adding Features

What if we use multiple variables to predict? Maybe volume tells us something the market alone doesn't.

It's just as easy:

In [ ]:
# Predict NVDA using market AND volume change
nvda = df[df["ticker"] == "NVDA"].copy().set_index("date")  # Get NVDA data
nvda["volume_chg"] = nvda["volume"].pct_change()        # Daily volume change

# Merge with market returns
data = nvda[["return", "volume_chg"]].join(returns["market"]).dropna()  # Combine features

X = data[["market", "volume_chg"]].values               # Two features (2D array)
y = data["return"].values                               # Target variable

model = LinearRegression().fit(X, y)                    # Fit with multiple features

print("Multiple Regression: NVDA ~ market + volume_chg")
print(f"Intercept: {model.intercept_:.6f}")
print(f"Coefficients:")
print(f"  market:     {model.coef_[0]:.4f}")            # First coefficient
print(f"  volume_chg: {model.coef_[1]:.6f}")            # Second coefficient
print(f"R²: {r2_score(y, model.predict(X)):.4f}")

print("\n→ Same pattern: create model, fit, predict. Works with any number of features.")
print("→ We'll use more features like this in Module 4.")

### ✏️ Checkpoint: Add Another Feature

Try adding `range_pct` (high - low / adj_close) as a third feature in the multiple regression. Does R² improve?

In [ ]:
# Your code here


---

## 4. Can We Predict Returns?

So far we've explained *today's* return using *today's* market. But what if we try to predict tomorrow's return using today's information?

Let's see how well a simple model can fit.

In [ ]:
# Prepare NVDA with features
nvda = df[df["ticker"] == "NVDA"].copy().set_index("date")  # Get NVDA data

# Features: things we know TODAY
nvda["return_today"] = nvda["return"]                   # Today's return
nvda["volume_chg"] = nvda["volume"].pct_change()        # Today's volume change
nvda["range_pct"] = (nvda["high"] - nvda["low"]) / nvda["adj_close"]  # Today's price range

# Target: TOMORROW's return (shift -1 = next row)
nvda["return_tomorrow"] = nvda["return"].shift(-1)      # What we're trying to predict

# Clean up
features = ["return_today", "volume_chg", "range_pct"]  # Our predictors
data = nvda[features + ["return_tomorrow"]].dropna()    # Drop rows with NaN

X = data[features].values                               # Feature matrix
y = data["return_tomorrow"].values                      # Target variable

print(f"Fitting on {len(data)} observations")
data.head()

In [ ]:
# Fit the model
model = LinearRegression().fit(X, y)                    # Fit on all data
y_pred = model.predict(X)                               # Get predictions

r2 = r2_score(y, y_pred)                                # How much variance explained?

print("=" * 50)
print("PREDICTING TOMORROW'S RETURN")
print("=" * 50)
print(f"R²: {r2:.4f}")
print("")
print("Coefficients:")
for feat, coef in zip(features, model.coef_):           # Show each feature's weight
    print(f"  {feat}: {coef:.6f}")

In [ ]:
# Visualize: Two views of the predictions
fig, axes = plt.subplots(1, 2, figsize=(14, 5))         # Side by side plots

# Left: Time series
axes[0].plot(data.index, y, label="Actual", alpha=0.7)  # Blue = actual returns
axes[0].plot(data.index, y_pred, label="Predicted", alpha=0.7)  # Orange = predictions
axes[0].axhline(0, color="gray", linestyle="--", alpha=0.5)
axes[0].set_xlabel("Date")
axes[0].set_ylabel("Return")
axes[0].set_title("Time Series: Predictions Are Basically Flat")
axes[0].legend()

# Right: Scatter plot with error coloring
errors = np.abs(y - y_pred)                             # Absolute prediction error
scatter = axes[1].scatter(y_pred, y, c=errors, cmap="viridis", alpha=0.6, s=15)

# Set x-axis to actual prediction range (with small buffer)
pred_min, pred_max = y_pred.min(), y_pred.max()         # Actual prediction range
buffer = (pred_max - pred_min) * 0.2                    # 20% buffer
axes[1].set_xlim(pred_min - buffer, pred_max + buffer)

# 45° reference line (across full y range)
lims = [min(axes[1].get_xlim()[0], axes[1].get_ylim()[0]),
        max(axes[1].get_xlim()[1], axes[1].get_ylim()[1])]
axes[1].plot(lims, lims, "k--", alpha=0.5, label="Perfect prediction")

axes[1].set_xlabel("Predicted Return")
axes[1].set_ylabel("Actual Return")
axes[1].set_title("Scatter: No Relationship")
axes[1].legend()
plt.colorbar(scatter, ax=axes[1], label="Prediction Error")

plt.suptitle(f"NVDA: Predicting Tomorrow's Return (R² = {r2:.4f})", fontsize=12)
plt.tight_layout()
plt.show()

print("Left: Predictions (orange) are nearly flat — the model just predicts ~0 every day.")
print("Right: If predictions worked, points would cluster along the diagonal. They don't.")

---

> **⚠️ Cautionary Tale: The Limits of Prediction**
>
> Look at that R²: **0.4%**. We can't even fit this regression to *existing* data.
>
> This isn't a coding error. We used real features (today's return, volume, price range) to predict tomorrow's return — and the model explains almost nothing. The predictions are essentially flat at zero.
>
> **If we can't fit historical data, we have no hope of forecasting future returns.**
>
> This is a central lesson in finance: **stock returns are very hard to predict.** Markets are efficient — if there were simple patterns, traders would exploit them until they disappeared. The signal-to-noise ratio in daily returns is tiny.
>
> Understanding this puts you ahead of people chasing get-rich-quick patterns.

### ✏️ Checkpoint: Try a Different Stock

Does NVDA predict any better than AAPL? Try changing the ticker.

In [ ]:
# Your code here


---

## 5. Exercises

Complete these to finish Module 3.

### Exercise 3.1: Rolling Beta

Beta isn't constant over time. Calculate NVDA's rolling 60-day beta and plot how it changes.

Hint: Loop through the data fitting a regression on each 60-day window.

In [ ]:
# Your code here


### Exercise 3.2: Add More Features

Try adding more features to the return prediction model:
- Lagged returns (return from 2 days ago, 3 days ago)
- Moving average signals
- Market return

Does the Test R² improve?

In [ ]:
# Your code here


### Exercise 3.3: Commit Your Work

```bash
git add .
git commit -m "Complete module 3 exercises"
git push
```

---

## Recap: Statistics with Scikit-Learn

| Concept | Code |
|---------|------|
| Correlation matrix | `df.corr()` |
| Create model | `model = LinearRegression()` |
| Fit model | `model.fit(X, y)` |
| Get coefficients | `model.intercept_`, `model.coef_` |
| Make predictions | `model.predict(X)` |
| Evaluate fit | `r2_score(y, y_pred)` |

### Key Takeaways

1. **Correlation** tells you if stocks move together
2. **Beta** measures a stock's sensitivity to the market
3. **R²** tells you how much variance your model explains
4. **Return prediction is hard** — this is a fundamental truth in finance

---

**Next up:** Module 4 — Machine Learning. We'll try classification and clustering.